## Интерфейс в виде телеграм-бота

Одним из хороших вариантов построения пользовательского интерфейса является использование чат-бота, например, в телеграм. Telegram API устроено таким образом, что позволяет обрабатывать сообщения в режиме поллинга, когда приложение запрашивает у Telegram API новые сообщения, и формирует на них ответы. В этом случае само приложение может находиться за межсетевым экраном, доступ к нему снаружи из интернет не требуется.

> Режим поллинга не рекомендуется для долгосрочных и высоконагруженных проектов, но для демонстрации прототипа вполне подойдёт.

Для реализации Telegram-бота можно использовать разные библиотеки, мы остановимся на `telebot`:

In [2]:
%pip install -q telebot


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


Для создания телеграм-бота необходимо использовать специального бота [botfather](http://t.me/botfather), как описано [тут](https://core.telegram.org/bots/features#creating-a-new-bot). Вы вводите команду `/newbot`, и после ответов на несколько простых вопросов получаете **токен** вот такого вида: `7292776300:AAGeoTIVI4pjNTxuCW7UckKcck5G1ZQVFi2d`. 

Далее мы будем предполагать, что токен содержится в DataSphere-секрете с именем `telegram_token`. 

В библиотеке `telebot` необходимо описать обработчики событий - это функции, которые будут вызываться в ответ на сообщения определённого вида. Например, для обработки команды `/start` мы описываем функцию с таким декоратором:

```python
@bot.message_handler(commands=['start'])
def start(message):
   ...
```

Мы реализуем бота, который будет классифицировать картинки. Для обработки сообщений, содержащих картинки, также используем соответствующий декоратор:

```python
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    ...
```

В конце мы запускаем функцию `bot.polling(none_stop=True)`, которая реализует бесконечный цикл обращений к Telegram API и обработки новых входящих сообщений. Запустив ячейчку вы можете перейти в телеграм-бота и начать взаимодействовать с ним.

In [6]:
import telebot
from PIL import Image
import tensorflow as tf
import numpy as np
import os,io

# API-ключ для доступа к телеграм-боту
telegram_token = os.environ['telegram_token']

# Создание экземпляра бота
bot = telebot.TeleBot(telegram_token)

# Загрузка модели классификации изображений
model = tf.keras.applications.MobileNetV2(weights="imagenet")

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    # Отправляем приветственное сообщение
    bot.send_message(message.chat.id,
                     'Привет, я бот, классифицирующий изображения. Пошлите мне какую-нибудь картинку!\n')

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    bot.send_message(message.chat.id, f"Привет, вы сказали {message.text}. Но лучше пошлите картинку!")

# Функция для классификации изображений.
def classify_image(image):
    image = tf.image.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0).copy()
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    preds = model.predict(image)
    _,l,c = tf.keras.applications.mobilenet_v2.decode_predictions(preds, top=1)[0][0]
    return l,c

# Обработчик сообщений с картинками
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    content = bot.download_file(file_info.file_path)
    image = Image.open(io.BytesIO(content))
    l,c = classify_image(image)
    bot.send_message(message.chat.id, f"Это {l} с вероятностью {c}.")

# Запуск бота
bot.polling(none_stop=True)

1/1 [==============================] - 1s 1s/step


Функция `polling` будет работать до тех пор, пока вы явно не прервёте выполнение ячейки, или DataSphere не сделает это за вас. Пока ячейка работает - вы можете продолжать общаться с ботом.

## Выводы

Telegram - это удобный способ создать пользовательский интерфейс для демонстрации функциональности вашего приложения или модели машинного обучения заказчикам, поскольку достаточно просто добавить бота себе в мессенджер. При этом возможет обмен текстом, файлами, изображениями и т.д. При этом демонстрационную версию бота можно запустить непосредственно в DataSphee.

Впоследствии, для более долгосрочного хостинга бота можно развернуть его на виртуальной машине, и перейти от режима поллинга к использованию WebHooks.